In [160]:
import numpy as np
import datetime as dt

In [182]:
alp = 10
cop = 10
problems = [[10,15,2,1,2], [20,20,3,3,4]]

In [183]:
def problem3(alp, cop, problems):
    problems = add_total_power(problems)
    problems = sorted(problems, key=lambda x: x[-1])
    
    solved = []
    time = 0
    while problems:
        solvable = check_solve_prob(alp, cop, problems)
        if solvable:
            for problem in solvable:
                problems.remove(problem)
                solved.append(problem)
                alp, cop, time = solve_problem(alp, cop, time, problem)
        else:
            if len(solved) == 0:
                alp, cop, time = increase_power(alp, cop, problems[0], time)
            else:
                choose, next_problem, solved_best = decide_solve_or_increase(alp, cop, solved, problems)
                if choose == 'solve':
                    problems.remove(next_problem)
                    solved.append(next_problem)
                    alp, cop, time = solve_problem(alp, cop, time, solved_best)
                else:
                    increase_power(alp, cop, next_problem, time)
                    
    return time

In [184]:
def decide_solve_or_increase(alp, cop, solved, problems):
    next_problem, diff_alp, diff_cop = choose_next_problem(alp, cop, problems)
    solved_best = choose_best_solved(diff_alp, diff_cop, solved)
    choose = compare_solve_increase(diff_alp, diff_cop, solved_best)
    
    return choose, next_problem, solved_best

In [185]:
def compare_solve_increase(diff_alp, diff_cop, solved_best):
    choose = ''
    alp_gap = max(diff_alp - solved_best[0], 0)
    cop_gap = max(diff_cop - solved_best[1], 0)
    
    if solved_best[4] <= (diff_alp+diff_cop-alp_gap-cop_gap):
        choose = 'solve'
    else:
        choose = 'increase'
    
    return choose

In [186]:
def choose_best_solved(diff_alp, diff_cop, solved):
    best = []
    for i, s in enumerate(solved):
        diff = diff_alp - s[0] + diff_cop - s[1]
        best.append([i, diff])
    best = sorted(best, key=lambda x: x[1])[0]
    
    return solved[best[0]]

In [198]:
def choose_next_problem(alp, cop, problems):
    ability_diff = []
    for i, prob in enumerate(problems):
        diff_alp = max(prob[0] - alp, 0)
        diff_cop = max(prob[1] - cop, 0)
        diff_sum = diff_alp + diff_cop
        ability_diff.append([diff_alp, diff_cop, diff_sum, i])
    next_problem = sorted(ability_diff, key=lambda x: x[2], reverse=True)[0]

    return problems[next_problem[3]], next_problem[0], next_problem[1]

In [199]:
def increase_power(alp, cop, problem, time):
    alp_need = problem[0] - alp
    cop_need = problem[1] - cop
    time = time + alp_need + cop_need
    
    return problem[0], problem[1], time

In [200]:
def solve_problem(alp, cop, time, problem):
    alp += problem[2]
    cop += problem[3]
    time += problem[4]
    
    return alp, cop, time

In [201]:
def check_solve_prob(alp, cop, problems):
    solvable = []
    for problem in problems:
        if (alp >= problem[0]) and (cop >= problem[1]):
            solvable.append(problem)
    
    return solvable

In [202]:
def add_total_power(problems):
    for problem in problems:
        problem.append(problem[0] + problem[1])
        
    return problems

In [203]:
problem3(alp, cop, problems)

9

#### Problem 5

In [40]:
rc1 = [[1,2,3], [4,5,6], [7,8,9]]
operations1 = ['Rotate', 'ShiftRow']

rc2 = [[1,2,3,4], [5,6,7,8], [9,10,11,12]]
operations2 = ['ShiftRow', 'Rotate', 'ShiftRow', 'Rotate']

In [109]:
def problem5(rc, operations):
    for operation in operations:
        if operation == 'Rotate':
            rc = rotate(rc)
        else:
            rc = shift(rc)
            
    return rc

def shift(rc):
    rc_new = []
    rc_new.append(rc[-1])
    rc_new.extend(rc[:-1])
    
    return rc_new

def rotate(rc):
    arr = np.array(rc)
    first_row = arr[0, :-1].copy()
    last_row = arr[-1, 1:].copy()
    first_col = arr[1:, 0].copy()
    last_col = arr[:-1, -1].copy()
    
    arr[0, 1:] = first_row
    arr[-1, :-1] = last_row
    arr[:-1, 0] = first_col
    arr[1:, -1] = last_col
    
    return arr.tolist()

In [110]:
problem5(rc1, operations1)

[[8, 9, 6], [4, 1, 2], [7, 5, 3]]

In [111]:
start = dt.datetime.now()
result = problem5(rc2, operations2)
end = dt.datetime.now()

print(result, end-start)

[[1, 6, 7, 8], [5, 9, 10, 4], [2, 3, 12, 11]] 0:00:00.000680


In [122]:
from collections import deque

def problem2(queue1, queue2):
    queue = queue1 + queue2
    total = sum(queue)
    if total % 2 != 0:
        return -1

    half = int(total / 2)
    q = deque(queue)

    elem_sum = 0
    cnt = 0
    tmp = deque()
    for i in range(len(q)):
        if i >= len(queue1):
            cnt += 1
        elem = q.popleft()
        tmp.append(elem)
        elem_sum += elem
        while elem_sum > half:
            move_elem = tmp.popleft()
            elem_sum -= move_elem
            q.append(move_elem)
            cnt += 1

        if elem_sum == half:
            return cnt

    return -1